### Complex Pheno/Geno workflow

This is the Complex Pheno/Geno workflow to be implemented as a web-based app using FLASK

In [2]:
from Bio import Entrez
from Settings import EMAIL # requires the presence of file Settings.py in current directory 

In [3]:
Entrez.email = EMAIL
# At Settings.py file: EMAIL = <username>@<something>.<something>
# This is the email you hav eused to register in NCBI

##### 1. Enter rsID
User is asked to enter the rsID that they know is associated with the complex disease of interest.
They are given the option to proceed with the analysis or to use PhenVar in order to check that the specific rsID is returning expected results (wordcloud and D3 graph) 

In [5]:
#get PMIDs from single rsID 

ID =input("Please enter an rsID number (do not include the letters 'rs'): ")

Please enter an rsID number (do not include the letters 'rs'): 800292


In [6]:
print('Do you want to proceed with analysis?')
print('Answering "no" will redirect to the PhenVar results for the given rsID. ')
print('Answering "yes" will take you to the next step where you will be required to enter VCF files. ')
conditional = input('Your answer: ')

Do you want to proceed with analysis?
Answering "no" will redirect to the PhenVar results for the given rsID. 
Answering "yes" will take you to the next step where you will be required to enter VCF files. 
Your answer: no


In [125]:
conditional = conditional.lower()
if conditional == 'no':
    url = 'https://phenvar.colorado.edu/results/?rsids='+ID+'&visualization=png-wordcloud&visualization=js-graph&normalization_type=default'
    print(url)
    
    import webbrowser
    webbrowser.open_new(url)
    #also works in Terminal as: python -m webbrowser -t "http://www.python.org" 
    
elif conditional != 'yes':
    print('Please go back and enter either "yes" or "no".')

https://phenvar.colorado.edu/results/?rsids=800292&visualization=png-wordcloud&visualization=js-graph&normalization_type=default


In [216]:
data =  Entrez.read(Entrez.elink(dbfrom='snp', db = 'pubmed', linkname='snp_pubmed_cited', id=ID))
pmids =  [id_dict['Id'] for id_dict in data[0]['LinkSetDb'][0]["Link"]]
print('There were {} PMIDs retrieved that are related to rs{}'.format(len(pmids), ID))

There were 128 PMIDs retrieved that are related to rs800292


In [220]:
rsids = []
for pmid in pmids:
    try:
        data = Entrez.read(Entrez.elink(dbfrom='pubmed',db='snp',linkname='pubmed_snp_cited',id=pmid))
        rsids.extend([id_dict['Id'] for id_dict in data[0]['LinkSetDb'][0]['Link']])
    except:
        print("There has been some problem with your connection. Please try again.")
        import sys
        sys.exit() #terminates all scripts running. To terminate only script that contains this try/except use quit()

In [385]:
#remove duplicates and turn dict to a list of all the rsIDs, regardless of how many publications support them
rsidSet = [id for id in set(rsids)]

print('A total of {} unique rsIDs were retrieved from {} PMIDs using rs{} as a search term\n'.format(len(rsidSet), len(pmids),ID))

A total of 757 unique rsIDs were retrieved from 128 PMIDs using rs800292 as a search term



##### 2. Filter retrieved rsIDs by number of supporting publications

In [398]:
#create a dictionary where the key is the number of publications supporting each rsID and values are the actual rsIDs
from collections import Counter
from Functions import rev_dict

In [399]:
rsDict = rev_dict(Counter(rsids)) 
 
#The following creates a list with the distinct numbers of supporting information in the results, ordered
numPmids = [i for i in set(rsDict.keys())]
numPmids.sort()

In [400]:
temp = rsiDict.copy()
#[item for sublist in temp.values() for item in sublist]
for i in numPmids:
    temp.pop(i,0)
    print('{} rsIDs are cited by more than {} PMIDs'.format(len([item for sublist in temp.values() for item in sublist]), i))

201 rsIDs are cited by more than 1 PMIDs
99 rsIDs are cited by more than 2 PMIDs
59 rsIDs are cited by more than 3 PMIDs
49 rsIDs are cited by more than 4 PMIDs
39 rsIDs are cited by more than 5 PMIDs
36 rsIDs are cited by more than 6 PMIDs
28 rsIDs are cited by more than 7 PMIDs
24 rsIDs are cited by more than 8 PMIDs
19 rsIDs are cited by more than 9 PMIDs
17 rsIDs are cited by more than 10 PMIDs
16 rsIDs are cited by more than 11 PMIDs
14 rsIDs are cited by more than 12 PMIDs
13 rsIDs are cited by more than 13 PMIDs
12 rsIDs are cited by more than 15 PMIDs
11 rsIDs are cited by more than 16 PMIDs
9 rsIDs are cited by more than 18 PMIDs
8 rsIDs are cited by more than 20 PMIDs
7 rsIDs are cited by more than 23 PMIDs
6 rsIDs are cited by more than 24 PMIDs
5 rsIDs are cited by more than 26 PMIDs
4 rsIDs are cited by more than 32 PMIDs
3 rsIDs are cited by more than 37 PMIDs
2 rsIDs are cited by more than 64 PMIDs
1 rsIDs are cited by more than 84 PMIDs
0 rsIDs are cited by more than 12

In [426]:
condition_filter = False
while condition_filter == False:
    print("What is the minimum acceptable number of publications supporting each rsID you want to consider?")
    print("Possible values are between 1 and {}.".format(max(numPmids[0:len(numPmids)-1])))
    no = int(input(''))
    if no <= max(numPmids[0:len(numPmids)-1]):
        selected_rsids = []
        for i in numPmids[::-1]:
            if i >= no:
                selected_rsids.extend(rsDict[i])
        selected_rsids = set(selected_rsids)
        condition_filter = True
    else:
        print("You have not entered a value between 1 and {}. Please retry.\n".format(max(numPmids[0:len(numPmids)-1])))
print('\nYou have selected to analyse {} different rsIDs that are co-cited together with rs{} (included).'.format(len(selected_rsids), ID))

What is the minimum acceptable number of publications supporting each rsID you want to consider?
Possible values are between 1 and 84.
2

You have selected to analyse 201 different rsIDs that are co-cited together with rs800292 (included).


In [424]:
selected_rsids = []
print(type(selected_rsids))
for i in numPmids[::-1]:
    selected_rsids.extend(rsDict[i])
    selected_rsids = set(selected_rsids)


<class 'list'>


AttributeError: 'set' object has no attribute 'extend'

In [422]:
temp = []
temp.extend(rsDict[2])
temp.extend(rsDict[2])
print(temp)

['2241392', '17030', '667604', '13207351', '12255372', '10759931', '7903146', '7901695', '6060566', '4986791', '3759890', '3134069', '2228570', '2073618', '1927914', '1801282', '1801133', '1800629', '1800592', '1799983', '1617640', '1544410', '1024611', '1002630', '854560', '833070', '833061', '660339', '551238', '507392', '361525', '39059', '7493', '5498', '662', '2075650', '1531289', '35507625', '2071559', '3790414', '876538', '2736912', '833069', '3811381', '2274567', '2876849', '1329424', '2284665', '10857341', '9542236', '4854022', '4711751', '3130783', '2014307', '1999930', '1831282', '1447338', '713586', '522162', '12678919', '10757278', '3880457', '3025033', '2808635', '2075702', '699946', '3812111', '433594', '482934', '11582939', '7517126', '1048663', '388862', '2672587', '2253755', '403846', '393955', '4788084', '4505848', '12124794', '7542235', '6663083', '1759016', '10486525', '10486523', '10486521', '10486519', '10254116', '4723261', '1420150', '1049024', '964707', '76412

In [362]:
# Export the identified rsIDs in a format (space delimited) that can be read by plink. 
#It will be used to extract the loci of interest from the VCFs
#Need to assign number for each user (no mix-ups) and arrange to delete at the end of the pipeline

with open('temp_rsids.txt', 'a+') as f:
    for item in selected_rsids:
        f.write("rs{} ".format(item))

##### 2. Upload VCFs and create patient - rsID matrix

I am currently working with the VCF for chr21 and chr22 (smallest autosomal) and chrY (doesn't have any rsIDs? Will it throw an error?) from [1000 genomes](ftp://ftp-trace.ncbi.nih.gov/1000genomes/ftp/release/20130502/) (downloaded on 10/13/2017)


In [26]:
from os import system, listdir
import re

In [38]:
#In Flask path will be the directory from which to upload. 
#A temp directory with the same name will be created in the server

path = input("Please enter the directory path where the vcf.gz files are located: ")

Please enter the directory path where the vcf.gz files are located: VCFs


In [84]:
# Create lists of all the uploaded VCFs and all the file prefixes (to be carried as file names during plink processing)

files = listdir(path)
if len(re.findall('.vcf.gz', str(files)))!=len(files): # checks that all files are .vcf.gz
    print("The files you uploaded are not in the <name>.vcf.gz format. Please check your uploaded folder and try again.")
else:
    prefix = [re.sub('.vcf.gz', '', file) for file in files]
#chrs 25 & 26 is chr22 duplicated. 

In [78]:
#The following function requires installation of plink 1.9 from https://www.cog-genomics.org/plink2 at the PATH directory

def ExtractRsID(Path, Prefix):
    print("Processing {}".format(Prefix))
    system('plink --vcf {}/{}.vcf.gz --recode12 --tab --extract temp_rsids.txt --out {}'.format(Path, Prefix,Prefix))

In [88]:
# Useful example: http://wltrimbl.github.io/2014-06-10-spelman/intermediate/python/04-multiprocessing.html
import multiprocessing
import progressbar

bar = progressbar.ProgressBar()
pool = multiprocessing.Pool(len(files)) # run as many processes as vcfs. The default is the number of CPU cores.
results = [pool.apply_async( ExtractRsID, (path, p) ) for p in prefix]
for result in bar(results):
    result.get()
#chr25 and chr26 is chr22 duplicated. The set of 3 chr (21,22,Y) took 1min6sec, set of 4 (= no. of cores) 1min18sec

## OK so this is not THAT fast (in my computer) perhaps using the faster cores at NCBI will help
## Implement some sort of timer in the web interface of the app
## http://www.java2s.com/Tutorial/Java/0240__Swing/Timerbasedanimation.htm

N/A% (0 of 5) |                          | Elapsed Time: 0:00:00 ETA:  --:--:--

Processing chr22
Processing chr21
Processing chrY
Processing chr25
Processing chr26


100% (5 of 5) |###########################| Elapsed Time: 0:01:51 Time: 0:01:51
Process ForkPoolWorker-42:
Process ForkPoolWorker-40:
Process ForkPoolWorker-43:
Process ForkPoolWorker-39:
Process ForkPoolWorker-41:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/local/Cellar/python3/3.6.2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/local/Cellar/python3/3.6.2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/local/Cellar/python3/3.6.2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/local/Cellar/python3/3.6.2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/process.py

In [113]:
#check for missing ped files

#files I have

#expected files

#make temp txt with ped files to merge

#here merge all files
#system('plink --file output.ped --recodeAD --out twostep')
